# Physical Variables

In [5]:
import autoroot
import typing as tp
from dataclasses import dataclass
import numpy as np
import pandas as pd
import xarray as xr
import einops
import xarray_dataclasses as xrdataclass
from oceanbench._src.datasets.base import XRDABatcher
from metpy.units import units

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
file = "/gpfswork/rech/cli/uvo53rl/projects/jejeqx/data/natl60/NATL60-CJM165_GULFSTREAM_ssh_y2013.1y.nc"
!ls $file

/gpfswork/rech/cli/uvo53rl/projects/jejeqx/data/natl60/NATL60-CJM165_GULFSTREAM_ssh_y2013.1y.nc


In [11]:
from xarray_dataclasses import Coordof, Dataof, Attr, Coord, Data, Name, AsDataArray, AsDataset, Dataof

TIME = tp.Literal["time"]
LAT = tp.Literal["lat"]
LON = tp.Literal["lon"]

@dataclass
class TimeAxis:
    data: Data[TIME, tp.Literal["datetime64[ns]"]]
    name: Name[str] = "time"
    long_name: Attr[str] = "Date"

@dataclass
class LatitudeAxis:
    data: Data[LAT, np.float32]
    name: Name[str] = "lat"
    standard_name: Attr[str] = "latitude"
    long_name: Attr[str] = "Latitude"
    units: Attr[str] = "degrees_west"

@dataclass
class LongitudeAxis:
    data: Data[LON, np.float32]
    name: Name[str] = "lon"
    standard_name: Attr[str] = "longitude"
    long_name: Attr[str] = "Longitude"
    units: Attr[str] = str(units.degrees_east)


@dataclass
class SSH2DT:
    data: Data[tuple[TIME, LAT, LON], np.float32]
    time: Coordof[TimeAxis] = 0
    lat: Coordof[LatitudeAxis] = 0
    lon: Coordof[LongitudeAxis] = 0
    name: Name[str] = "ssh"
    units: Attr[str] = "m"
    standard_name: Attr[str] = "sea_surface_height"
    long_name: Attr[str] = "Sea Surface Height"

In [15]:

lon = np.linspace(-65, -54, 50)
lat = np.linspace(32, 44, 50)
tmin, tmax = pd.to_datetime("2013-01-01"), pd.to_datetime("2013-12-31")
dt = pd.to_timedelta(1, "D")
time = np.arange(tmin, tmax, dt)
rng = np.random.RandomState(seed=123)

data = rng.randn(time.shape[0], lat.shape[0], lon.shape[0])

grid = SSH2DT(data=data, lat=lat, lon=lon, time=time)

da = xrdataclass.asdataarray(grid).to_dataset()

da

<xarray.Dataset>
Dimensions:  (time: 364, lat: 50, lon: 50)
Coordinates:
  * time     (time) datetime64[ns] 2013-01-01 2013-01-02 ... 2013-12-30
  * lat      (lat) float32 32.0 32.24 32.49 32.73 ... 43.27 43.51 43.76 44.0
  * lon      (lon) float32 -65.0 -64.78 -64.55 -64.33 ... -54.45 -54.22 -54.0
Data variables:
    ssh      (time, lat, lon) float32 -1.086 0.9973 0.283 ... -0.8167 0.8466

In [56]:
da.lon

<xarray.DataArray 'lon' (lon: 50)>
array([-65.      , -64.77551 , -64.55102 , -64.32653 , -64.10204 , -63.877552,
       -63.65306 , -63.42857 , -63.204082, -62.97959 , -62.7551  , -62.530613,
       -62.30612 , -62.081635, -61.857143, -61.632652, -61.408165, -61.183674,
       -60.959183, -60.734695, -60.510204, -60.285713, -60.061226, -59.836735,
       -59.612244, -59.387756, -59.163265, -58.938774, -58.714287, -58.489796,
       -58.265305, -58.040817, -57.816326, -57.591835, -57.367348, -57.142857,
       -56.918365, -56.69388 , -56.469387, -56.2449  , -56.02041 , -55.795918,
       -55.57143 , -55.34694 , -55.122448, -54.89796 , -54.67347 , -54.44898 ,
       -54.22449 , -54.      ], dtype=float32)
Coordinates:
  * lon      (lon) float32 -65.0 -64.78 -64.55 -64.33 ... -54.45 -54.22 -54.0

## Degrees to Meters

In [81]:
from oceanbench._src.geoprocessing.spatial import latlon_deg2m
from oceanbench._src.geoprocessing.temporal import time_rescale

In [83]:
# out_da = latlon_deg2m(da, mean=False)
out_da = time_rescale(da, 1, "s")
out_da

<xarray.Dataset>
Dimensions:  (time: 365, lat: 201, lon: 201)
Coordinates:
  * lon      (lon) float64 -65.0 -64.95 -64.9 -64.85 ... -55.1 -55.05 -55.0
  * lat      (lat) float64 33.0 33.05 33.1 33.15 33.2 ... 42.85 42.9 42.95 43.0
  * time     (time) float64 0.0 8.64e+04 1.728e+05 ... 3.136e+07 3.145e+07
Data variables:
    ssh      (time, lat, lon) float64 ...

In [27]:
da.lon

<xarray.DataArray 'lon' (lon: 50)>
array([-65.      , -64.77551 , -64.55102 , -64.32653 , -64.10204 , -63.877552,
       -63.65306 , -63.42857 , -63.204082, -62.97959 , -62.7551  , -62.530613,
       -62.30612 , -62.081635, -61.857143, -61.632652, -61.408165, -61.183674,
       -60.959183, -60.734695, -60.510204, -60.285713, -60.061226, -59.836735,
       -59.612244, -59.387756, -59.163265, -58.938774, -58.714287, -58.489796,
       -58.265305, -58.040817, -57.816326, -57.591835, -57.367348, -57.142857,
       -56.918365, -56.69388 , -56.469387, -56.2449  , -56.02041 , -55.795918,
       -55.57143 , -55.34694 , -55.122448, -54.89796 , -54.67347 , -54.44898 ,
       -54.22449 , -54.      ], dtype=float32)
Coordinates:
  * lon      (lon) float32 -65.0 -64.78 -64.55 -64.33 ... -54.45 -54.22 -54.0
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east

In [28]:
from metpy.units import units

In [29]:
units.degrees

<Unit('degree')>

In [ ]:
units.